In [1]:
from os import listdir
from os.path import isfile, join
import plotly.express as px
import regex as re
import pandas as pd
from pandas import DataFrame
from pandas import to_datetime
import matplotlib 
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot, plot_cross_validation_metric
from fbprophet.diagnostics import cross_validation, performance_metrics

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
path = '/Users/mattiaficarelli/Desktop/AE_admissions_data/'
file_header_list = ['Period',
'Org Code', 
'Parent Org', 
'Org name', 
'A&E attendances Type 1', 
'A&E attendances Type 2', 
'A&E attendances Other A&E Department',
'A&E attendances Booked Appointments Type 1',
'A&E attendances Booked Appointments Type 2',
'A&E attendances Booked Appointments Other Department',
'Attendances over 4hrs Type 1',
'Attendances over 4hrs Type 2',
'Attendances over 4hrs Other Department',
'Attendances over 4hrs Booked Appointments Type 1',
'Attendances over 4hrs Booked Appointments Type 2',
'Attendances over 4hrs Booked Appointments Other Department',
'Patients who have waited 4-12 hs from DTA to admission',
'Patients who have waited 12+ hrs from DTA to admission',
'Emergency admissions via A&E - Type 1',
'Emergency admissions via A&E - Type 2',
'Emergency admissions via A&E - Other A&E department',
'Other emergency admissions']


In [3]:
path_excel = '/Users/mattiaficarelli/Desktop/AE_admissions_data_prior_April_2018/'
file_header_list_excel = ['Code', 
'Region', 'Name', 
'Type 1 Departments - Major A&E', 
'Type 2 Departments - Single Specialty', 
'Type 3 Departments - Other A&E/Minor Injury Unit']

In [4]:

onlyexcelfiles = [f for f in listdir(path_excel) if isfile(join(path_excel, f))]
df_append_excel = pd.DataFrame()
regex_to_extract_date = re.compile(r'^([^.]*)-AE*')
for file in onlyexcelfiles:
    df = pd.read_excel(path_excel + file, names = file_header_list_excel, skiprows=17, usecols="B:G")
    date_from_filename = regex_to_extract_date.findall(file)
    df['Date']= date_from_filename[0]
    df_append_excel = df_append_excel.append(df, ignore_index=True)

In [5]:
df_append_excel_clean_1 = df_append_excel[df_append_excel['Code'].notna()].reset_index(drop = True)
df_append_excel_clean_1['Date'] = pd.to_datetime(df_append_excel_clean_1['Date'])
df_append_excel_clean_2 = df_append_excel_clean_1.rename(columns = {'Code': 'ODS code', 'Name': 'Organisation name', 'Type 1 Departments - Major A&E': 'Type 1', 'Type 2 Departments - Single Specialty': 'Type 2', 'Type 3 Departments - Other A&E/Minor Injury Unit': 'Other'})
df_append_excel_clean_3 = df_append_excel_clean_2[["Date", "ODS code", "Organisation name", "Type 1", "Type 2", "Other"]]
df_append_excel_clean_3["Total A&E attendences"] = df_append_excel_clean_3["Type 1"] + df_append_excel_clean_3["Type 2"] + df_append_excel_clean_3["Other"]
df_append_excel_clean_4 = df_append_excel_clean_3.sort_values(by=['Date']).reset_index(drop = True)
df_append_excel_clean_4

,Date,ODS code,Organisation name,Type 1,Type 2,Other,Total A&E attendences
0,2015-06-01,RBT,Mid Cheshire Hospitals NHS Foundation Trust,4958.0,0.0,1974.0,6932.0
1,2015-06-01,RM3,Salford Royal NHS Foundation Trust,8039.0,0.0,0.0,8039.0
2,2015-06-01,RWJ,Stockport NHS Foundation Trust,7835.0,0.0,0.0,7835.0
3,2015-06-01,RMP,Tameside Hospital NHS Foundation Trust,7026.0,0.0,0.0,7026.0
4,2015-06-01,RBV,The Christie NHS Foundation Trust,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
8290,2018-03-01,RTR,South Tees Hospitals NHS Foundation Trust,10501.0,0.0,1761.0,12262.0
8291,2018-03-01,NQT5F,Skelmersdale Walk In Centre,0.0,0.0,1783.0,1783.0
8292,2018-03-01,RHQ,Sheffield Teaching Hospitals NHS Foundation Trust,9314.0,1097.0,7596.0,18007.0
8293,2018-03-01,AXT02,Shakespeare Walk-In Centre,0.0,0.0,3945.0,3945.0


In [6]:

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
df_append = pd.DataFrame()
for file in onlyfiles:
    df = pd.read_csv(path + file, names = file_header_list, skiprows=1)
    df_append = df_append.append(df, ignore_index=True)

In [7]:
df_clean_1 = df_append[df_append["Period"].str.contains("TOTAL")==False].reset_index(drop = True)
df_clean_1['Period'] = pd.to_datetime(df_clean_1['Period'].str.replace("MSitAE-", ""))
df_clean_2 = df_clean_1.sort_values(by=['Period']).reset_index(drop = True)
df_clean_3 = df_clean_2[["Period", "Org Code","Org name", "A&E attendances Type 1","A&E attendances Type 2", 'A&E attendances Other A&E Department']]
df_clean_4 = df_clean_3.rename(columns = {'Period': 'Date', 'Org Code': 'ODS code', 'Org name': 'Organisation name', 'A&E attendances Type 1': 'Type 1', 'A&E attendances Type 2': 'Type 2', 'A&E attendances Other A&E Department': 'Other'})
df_clean_4["Total A&E attendences"] = df_clean_4["Type 1"] + df_clean_4["Type 2"] + df_clean_4["Other"]

In [8]:
df_final = df_append_excel_clean_4.append(df_clean_4)
df_final_1 = df_final.drop(["ODS code", "Organisation name"], axis=1)
df_final.to_csv('final_cleaned_data.csv')

In [9]:
df_final_group = df_final_1.groupby("Date").sum().reset_index()
df_final_group

,Date,Type 1,Type 2,Other,Total A&E attendences
0,2015-06-01,1249213.0,53398.0,608598.0,1911209.0
1,2015-07-01,1271523.0,53417.0,627955.0,1952895.0
2,2015-08-01,1215826.0,49458.0,599855.0,1865139.0
3,2015-09-01,1221594.0,50868.0,587517.0,1859979.0
4,2015-10-01,1261395.0,50275.0,611438.0,1923108.0
...,...,...,...,...,...
74,2021-08-01,1307515.0,39577.0,619694.0,1966786.0
75,2021-09-01,1359673.0,40228.0,659084.0,2058985.0
76,2021-10-01,1384402.0,39474.0,665990.0,2089866.0
77,2021-11-01,1300577.0,38821.0,624035.0,1963433.0


In [10]:
import os

In [16]:
pd.options.plotting.backend = "plotly"
fig = px.bar(df_final_group, x='Date', y= ["Type 1", "Type 2","Other"],
color_discrete_sequence=["#003087", "#0072CE", "#41B6E6"],
labels={"value": "Number of A&E attendances", "variable": "A&E attendances"})
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    font = dict(family = "Arial", size = 16),
    autosize=True,
    margin=dict(l=50, r=50, b=50, t=50, pad=4, autoexpand=True),
)
fig.show()
import plotly.io as pio
pio.write_image(fig, 'image.png', format= "png", scale=1, width=1500, height=700)

In [ ]:
mask = (df_final_group['Date'] <= '2019-10-01')
df_final_precovid = df_final_group.loc[mask]
df_final_precovid 

In [ ]:
df_precvoid_prophet = df_final_precovid[['Date', 'Total A&E attendences']]
df_precvoid_prophet_1 = df_precvoid_prophet.rename(columns = {'Date': 'ds', 'Total A&E attendences': 'y' })

In [ ]:
model = Prophet()
model.fit(df_precvoid_prophet_1)
future = list()
for i in range(1, 13):
	date = '2019-%01d-01' % i
	future.append([date])
future = DataFrame(future)
future.columns = ['ds']
future['ds']= to_datetime(future['ds'])
forecast = model.predict(future)
# summarize the forecast
historical_forecast = pd.DataFrame(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])
# plot forecast
figure(figsize = (18, 14), dpi = 250)
model.plot(forecast)
matplotlib.pyplot.xlabel("Date")
matplotlib.pyplot.ylabel("Total A&E attendences (x10^6)")
plt.title("Total A&E attendences Prophet Prediction fit 2019-01 to 2019-12", fontweight ="bold")
pyplot.show()
plt.rcParams.update({'font.size': 16})

In [ ]:
figure(figsize = (18, 14), dpi = 250)
future = list()
for i in range(1, 13):
	date = '2020-%01d' % i
	future.append([date])
for i in range(1, 13):
  date = '2021-%01d' % i
  future.append([date])
future = DataFrame(future)
future.columns = ['ds']
future['ds']= to_datetime(future['ds'])
forecast = model.predict(future)
# summarize the forecast
future_forecast = pd.DataFrame(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])
# plot forecast
model.plot(forecast)
matplotlib.pyplot.xlabel("Date")
matplotlib.pyplot.ylabel("Total A&E attendences (x10^6)")
plt.title("Total A&E attendences Prophet Prediction 2020-01 to 2021-12", fontweight ="bold")
pyplot.show()
plt.rcParams.update({'font.size': 16})


In [ ]:
#Manually Evaluate Forecast Model
train = df_precvoid_prophet_1.drop(df_precvoid_prophet_1.index[-12:])
print(train)
# define the model
model = Prophet()
# fit the model
model.fit(train)
# define the period for which we want a prediction
figure(figsize = (18, 14), dpi = 250)
future = list()
for i in range(1, 13):
	date = '2019-%01d-01' % i
	future.append([date])
future = DataFrame(future)
future.columns = ['ds']
future['ds'] = to_datetime(future['ds'])
# use the model to make a forecast
forecast = model.predict(future)
# calculate MAE between expected and predicted values for december
y_true = df_precvoid_prophet_1['y'][-12:].values
y_pred = forecast['yhat'].values
# plot expected vs actual
pyplot.plot(y_true, label='Actual')
pyplot.plot(y_pred, label='Predicted')
matplotlib.pyplot.xlabel("Month")
matplotlib.pyplot.ylabel("Total A&E attendences (x10^6)")
plt.title("Plot of Actual vs. Predicted Values for Last 11 Months of Total A&E attendence data", fontweight ="bold")
pyplot.legend()
pyplot.show()
plt.rcParams.update({'font.size': 16})

In [ ]:
mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)